# pdf2reveal
This notebook transforms a presentation PDF made using LibreOffice Impress, Microsoft Powerpoint or Latex Beamer into HTML (reveal.js). This allows displaying the presentation on any device running a web browser. 

I wrote this so that I can present lecture slides on a Promethean without having to connect my computer. 

### Technical details

###### Initialization

During the initial run of this notebook, the **install_reveal** function installed the latest version of **reveal.js** which was **4.1.3**. The **create_template** function then created a **jinja2** template by editing 
**reveal.js-master/index.html**. Note that my notebook code may break if a later version of **reveal.js** is installed.

###### GitHub repository
A copy of this notebook and all files and directories created by it is maintained in a GitHub repository at https://github.com/aubreymoore/albi345-slides. This repo serves GitHub pages at https://aubreymoore.github.io/albi345-slides/. 

###### Example usage
```
papermill \ 
-p PDFPATH /home/aubrey/ALBI-345/files/pdfs/Orders-with-Complete-Metamorphosis.pdf \ 
pdf2reveal.ipynb \ 
out.ipynb
```
After running this example and pushing the local repo to gitHub, the online HTML (reveal.js) presentation will be available at 
https://aubreymoore.github.io/albi345-slides/Orders-with-Complete-Metamorphosis

###### Example bash script
```bash
#!/bin/bash
echo $1
if test -r $1
then
    papermill -p PDFPATH $1 pdf2reveal.ipynb out.ipynb
	git pull
	git add .
	git commit -m "$1"
	git push
else
    echo "ERROR: Cannot find PDF at $1"
fi
```

###### Directory structure:
```
pdf2reveal.ipynb          This notebook.
index.html                Recreated every time this notebook is run. Index of presentations.
template.html             Created during intitial run of this notebook.
out.ipynb                 Created when running this notebook using papermill. Useful for error checking.
reveal.js-master (dir)    Created during intitial run of this notebook.
images (dir)              Created during intitial run of this notebook.
   presentation1-000.jpg  Created when PDFPATH is "*/presentation1.pdf"
   presentation1-001.jpg  Created when PDFPATH is "*/presentation1.pdf"
   ...
   presentation2-000.jpg  Created when PDFPATH is "*/presentation2.pdf"
   presentation2-001.jpg  Created when PDFPATH is "*/presentation2.pdf"
   ...
presentation1 (dir)       Created when PDFPATH is "*/presentation1.pdf"
   presentation1.pdf      Created when PDFPATH is "*/presentation1.pdf" Copy of original PDF.
   index.html             Created when PDFPATH is "*/presentation1.pdf" This is the HTML presentation.  
presentation2 (dir)       Created when PDFPATH is "*/presentation2.pdf"
   presentation2.pdf      Created when PDFPATH is "*/presentation2.pdf" Copy of original PDF.
   index.html             Created when PDFPATH is "*/presentation2.pdf" This is the HTML presentation.
```

##### Fixing imagemagick problems
This notebook uses the **convert** command, which is part of **imagemagick** to convert PDFs to images.
Had some initial problems which were fixed by changing a few lines in the config file **/etc/ImageMagick-6/policy.xml**:
```
<policy domain="coder" rights="none" pattern="PDF" />
<policy domain="resource" name="memory" value="256MiB"/>
<policy domain="resource" name="disk" value="1GiB"/>
```
where changed to:
```
<!--  <policy domain="coder" rights="none" pattern="PDF" />  -->
<policy domain="resource" name="memory" value="8GiB"/>
<policy domain="resource" name="disk" value="8GiB"/>
```
### References
* https://marcus-baw.medium.com/using-reveal-js-a74b30e4065b
* https://martinomensio.medium.com/how-to-host-reveal-js-slides-on-github-pages-and-have-a-tidy-repository-1a363944c38d
* https://superuser.com/questions/1178666/imagemagick-convert-quits-after-some-pages

In [1]:
import os
import requests, zipfile, io
import jinja2
import glob
import shutil

In [2]:
# Parameters which may be set using papermill
PDFPATH = '/home/aubrey/Downloads/SWCD-2021-07-30.pdf'
GITHUB_REPO='https://github.com/aubreymoore/albi345-slides'
GITHUB_PAGES='https://aubreymoore.github.io/albi345-slides'

In [3]:
# Parameters
PDFPATH = "/home/aubrey/ALBI-345/files/pdfs/urban-entomology.pdf"


In [4]:
def install_reveal():
    """
    Download and install the latest version of reveal.js.
    """
    if not os.path.exists('reveal.js-master'):
        r = requests.get('https://github.com/hakimel/reveal.js/archive/master.zip')
        z = zipfile.ZipFile(io.BytesIO(r.content))
        z.extractall()

In [5]:
def create_template():
    """
    Create a jinja2 template by editing the reveal.js default index.html
    """

    # Read the default index.html as a string    
    s = open('reveal.js-master/index.html', 'r').read()

    # edit the string and save it as template.html
    s = s.replace('"dist', '"../reveal.js-master/dist')
    s = s.replace('"plugin', '"../reveal.js-master/plugin')
    s = s.replace('<section>Slide 1</section>\n\t\t\t\t<section>Slide 2</section>',
              '{% for image in images %}\n\t\t\t\t\t<section> <img src="../{{ image }}"> </section>\n\t\t\t\t{% endfor %}')

    with open('template.html', 'w') as f:
        f.write(s)

In [6]:
def create_presentation():
    """
    Use the jinja2 template to add slides from the images directory
    """
    
    if not os.path.exists('images'):
        os.mkdir('images')    
    
    # Create a directory for the presentation and put a copy of the PDF in it
    pdf_name = os.path.basename(PDFPATH).replace('.pdf', '')
    os.mkdir(pdf_name)
    shutil.copyfile(PDFPATH, f'{pdf_name}/{pdf_name}.pdf')
    
    # Convert the PDF pages into jpg images and store them in the communal images directory
    os.system(f'convert -density 300 {PDFPATH} images/{pdf_name}-%03d.jpg')
    images = sorted(glob.glob(f'images/{pdf_name}-???.jpg'))  
    print(images)
       
    templateLoader = jinja2.FileSystemLoader(searchpath="./")
    templateEnv = jinja2.Environment(loader=templateLoader)
    TEMPLATE_FILE = "template.html"
    template = templateEnv.get_template(TEMPLATE_FILE)
    outputText = template.render({'images': images})

    with open(f'{pdf_name}/index.html', 'w') as out:
        out.write(outputText)

In [7]:
def update_main_index():
    """
    Update the main GitHub pages index
    """
    # Get a list of presentations from the image directory 
    images = glob.glob('images/*.jpg')
    for i, image in enumerate(images):
        images[i] = image[7:-8]
    presentations = set(images)
    presentations = list(presentations)
    presentations = sorted(presentations, key=str.casefold)

    # Create index page
    s = '<h1>HTML (reveal.js) presentations for ALBI-345</h1>\n'
    for presentation in presentations:
        s += f'<a href="{GITHUB_PAGES}/{presentation}">{presentation}</a><br>\n'
    with open("index.html", "w") as f:
        f.write(s)

In [8]:
# MAIN

PRESENTATION = os.path.basename(PDFPATH).replace('.pdf', '')
if not os.path.exists('reveal.js-master'):
    install_reveal()
    create_template()
create_presentation()
update_main_index()
print(f'Finished converting {PDFPATH}')
print('Do not forget to update git and push to GitHub.')

['images/urban-entomology-000.jpg', 'images/urban-entomology-001.jpg', 'images/urban-entomology-002.jpg', 'images/urban-entomology-003.jpg', 'images/urban-entomology-004.jpg', 'images/urban-entomology-005.jpg', 'images/urban-entomology-006.jpg', 'images/urban-entomology-007.jpg', 'images/urban-entomology-008.jpg', 'images/urban-entomology-009.jpg', 'images/urban-entomology-010.jpg', 'images/urban-entomology-011.jpg', 'images/urban-entomology-012.jpg', 'images/urban-entomology-013.jpg', 'images/urban-entomology-014.jpg', 'images/urban-entomology-015.jpg', 'images/urban-entomology-016.jpg', 'images/urban-entomology-017.jpg', 'images/urban-entomology-018.jpg', 'images/urban-entomology-019.jpg', 'images/urban-entomology-020.jpg', 'images/urban-entomology-021.jpg', 'images/urban-entomology-022.jpg', 'images/urban-entomology-023.jpg', 'images/urban-entomology-024.jpg', 'images/urban-entomology-025.jpg', 'images/urban-entomology-026.jpg', 'images/urban-entomology-027.jpg', 'images/urban-entom